In [5]:
# Librerías
import pandas as pd
from sklearn.model_selection import train_test_split
# Becnhmark models
from sklearn import svm
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
# Escalamiento de datos
from sklearn.preprocessing import StandardScaler
# Warnings
import warnings
warnings.filterwarnings('ignore')
# Internal tool and helpers
import model_helpers as mh

In [3]:
data_path = "../CHESS/data/df_2_cod.csv"
df_2_cod = pd.read_csv(data_path)
df_2_cod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20058 entries, 0 to 20057
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   game_id                20058 non-null  int64 
 1   turns                  20058 non-null  int64 
 2   white_rating           20058 non-null  int64 
 3   black_rating           20058 non-null  int64 
 4   moves                  20058 non-null  object
 5   opening_moves          20058 non-null  int64 
 6   ctrld_pawn             20058 non-null  int64 
 7   ctrld_knight           20058 non-null  int64 
 8   ctrld_bishop           20058 non-null  int64 
 9   ctrld_rook             20058 non-null  int64 
 10  ctrld_queen            20058 non-null  int64 
 11  ctrld_king             20058 non-null  int64 
 12  preassure_points       20058 non-null  int64 
 13  controlled_diagonals   20058 non-null  int64 
 14  controlled_lines       20058 non-null  int64 
 15  rated_cod          

In [8]:
# X e Y 
df = df_2_cod.copy()
y_name = "winner_cod"
# X es el dataframe eliminando la variable de salida. Eliminando también 'moves' que ya está representado
X = df.drop(columns=[y_name, 'moves'])
# Y es un array unidimensional (ravel) de la variable de salida
Y = df[y_name].ravel()


# Escalamiento de los datos
El StandardScaler es una técnica de preprocesamiento de datos en aprendizaje automático que se utiliza para estandarizar las características (variables de entrada) escalándolas de tal manera que tengan una media (promedio) de 0 y una desviación estándar de 1.

Algunos algoritmos de aprendizaje automático, especialmente aquellos que involucran distancias (como K-Means, SVM, y KNN) o asumen una distribución normal (como regresión logística, LDA), se benefician de la estandarización de las características.

Cuando se tienen columnas con muchas clases, se incrementa el tiempo de ejecución por mucho esto se ve todavía más grabe con el Kernel lineal.
Por esta razon se aplica un escalamiento estandar.

Respecto a XGBOOST, no suele ser necesario y no tiene un gran impacto en el rendimiento del modelo porque es invariante a la escala de las características. 

In [9]:
# Escalamiento y división en train y test
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X),columns = X.columns)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size=0.3)

# Maquina de soporte vectorial

In [11]:
# Crear un clasificador SVM para clasificación con kernel de base radial
mod_rbf = svm.SVC(kernel='rbf',C=1,gamma='auto')
mod_rbf.fit(X_train, Y_train)

SVC(C=1, gamma='auto')

In [12]:
Yhat_rbf_test = mod_rbf.predict(X_test)
Yhat_rbf_train = mod_rbf.predict(X_train)
mh.eval_perform_class(Y_test,Yhat_rbf_test, "Prueba", True)
mh.eval_perform_class(Y_train,Yhat_rbf_train, "Entrenamiento", True)


Performance del modelo de Prueba
Accu 0.8688933200398804 
 Prec 0.8510976792205357 
 Reca 0.8688933200398804

Performance del modelo de Entrenamiento
Accu 0.888960113960114 
 Prec 0.8939622361874665 
 Reca 0.888960113960114


(0.888960113960114, 0.8939622361874665, 0.888960113960114)

# XGboost

In [13]:
# Crear un clasificador XGBoost
xgboost = xgb.XGBClassifier()
# Entrenar el modelo en los datos de entrenamiento
xgboost.fit(X_train, Y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [14]:
Yhat_xgboost_test = xgboost.predict(X_test)
Yhat_xgboost_train = xgboost.predict(X_train)
mh.eval_perform_class(Y_test,Yhat_xgboost_test, "Prueba", True)
mh.eval_perform_class(Y_train,Yhat_xgboost_train, "Entrenamiento", True)


Performance del modelo de Prueba
Accu 0.8772017281488866 
 Prec 0.8698236947016245 
 Reca 0.8772017281488866

Performance del modelo de Entrenamiento
Accu 0.9878205128205129 
 Prec 0.9878922903623923 
 Reca 0.9878205128205129


(0.9878205128205129, 0.9878922903623923, 0.9878205128205129)

# Regresión logística

In [15]:
# Regularización L1 (Lasso)
reg_log = LogisticRegression(penalty='l1', solver='liblinear', max_iter=1000)
reg_log.fit(X_train,Y_train)

LogisticRegression(max_iter=1000, penalty='l1', solver='liblinear')

In [16]:
Yhat_log_test = reg_log.predict(X_test)
Yhat_log_train = reg_log.predict(X_train)
mh.eval_perform_class(Y_test,Yhat_log_test, "Prueba", True)
mh.eval_perform_class(Y_train,Yhat_log_train, "Entrenamiento", True)


Performance del modelo de Prueba
Accu 0.8517779993353274 
 Prec 0.8474827911056829 
 Reca 0.8517779993353274

Performance del modelo de Entrenamiento
Accu 0.8565527065527065 
 Prec 0.84222834883223 
 Reca 0.8565527065527065


(0.8565527065527065, 0.84222834883223, 0.8565527065527065)